<a href="https://colab.research.google.com/github/siddhesh-k/Free-QR-Code-Generator-App/blob/main/Deepfake_end_to_end_tortoise_wav2lip_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Automatic DeepFake Creation (Tortoise voice cloning + wav2lip)

Note: A video is required. If an audio is also provided, the voice will be cloned from the audio.

# **The video should have a face looking at all times to the camera.**

## **Video (and audio) file should be in Google drive in a folder named 'deepfake'. No other files should exist there**

wav2lip code taken from https://github.com/snehitvaddi/Deepfake-using-Wave2Lip

In [1]:
#@title Upload video.mp4 (video to overlay voice) & voice.mp3 (voice to clone) files - Should be mp3 and mp4, having any name
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/deepfake

/content/gdrive/MyDrive/deepfake


In [3]:
base_path='/content/gdrive/MyDrive/deepfake' #Specify path of video/audio

In [ ]:
#@title Install TTS, pydub to create folders with audio chunks, and moviepy to modify duration of audio/video
!pip uninstall -y numpy scipy librosa TTS
!pip install scipy librosa TTS soundfile
!pip install -q pydub moviepy==1.0.3

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
Found existing installation: librosa 0.11.0
Uninstalling librosa-0.11.0:
  Successfully uninstalled librosa-0.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 78.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 129.3 MB/s eta 0:00:00
  Preparing metadata (set

In [ ]:
#@title English text that we want to read with the cloned voice - This will be inserted in the video too
#Text prompt should be separated with '|' every one to two sentences (every ~20secs it takes to read it).
text_to_read="""Joining two modalities results in a surprising increase in generalization! |
                What would happen if we combined them all? """

In [ ]:
#@title Rename audio and video files to be used below
#Prompt: the user selects and uploads to google colab one audio and one video file. Rename the audio file to 'input_voice.mp3' and the video to 'input_video.mp4'
import os

# Loop over files in the directory
for file in os.listdir(os.getcwd()):

      filename = os.path.join(base_path, file)

      if filename.endswith('.mp3'):
          new_filename = 'input_voice.mp3'
          os.rename(filename, new_filename)
      if filename.endswith('.mp4'):
          new_filename = 'video_full.mp4'
          os.rename(filename, new_filename)

#If only video is provided:
from moviepy.editor import VideoFileClip

def extract_audio(input_video, output_audio):
    video = VideoFileClip(input_video)
    audio = video.audio
    audio.write_audiofile(output_audio)

# Provide the input video file path and desired output audio file path
input_video = 'video_full.mp4'
output_audio = 'input_voice.mp3'

#Decide if voice will be cloned from video or audio
mp3_check=0
for file in os.listdir(os.getcwd()):
      file_path =  os.path.join(base_path, file)
      if '.mp3' in file_path:
        mp3_check=1

if mp3_check==0:
  print("Voice will be cloned from video")
  extract_audio(input_video, output_audio)

In [ ]:
#@title Create folder with 10 secs chunks of audio to be used as input in Tortoise
from pydub import AudioSegment

def split_audio_to_clips(audio_file, output_dir, clip_length=10000, sample_rate=22050):
    # Load the audio file
    audio = AudioSegment.from_mp3(audio_file)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Calculate the total number of clips
    num_clips = len(audio) // clip_length

    # Split the audio into clips and save them as WAV files
    for i in range(num_clips):
        start_time = i * clip_length
        end_time = start_time + clip_length
        clip = audio[start_time:end_time]

        # Set the sample width to 2 bytes for floating-point format
        clip = clip.set_sample_width(2)

        # Set the sample rate to 22050 Hz
        clip = clip.set_frame_rate(sample_rate)

        # Save the clip as a WAV file
        clip.export(os.path.join(output_dir, f"{i+1}.wav"), format="wav")

    print(f"{num_clips} clips saved in '{output_dir}'.")

if __name__ == "__main__":

    # Replace 'input_audio.mp3' with the name of your MP3 file
    input_audio_file = base_path+'/input_voice.mp3'

    # Replace 'voices' with the desired subdirectory name
    subdirectory_name = base_path+'/voice'

    split_audio_to_clips(input_audio_file, subdirectory_name)

In [ ]:


!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.31.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

In [ ]:
#@title Download and run TTS tortoise model
import librosa
from TTS.api import TTS
tts = TTS("tts_models/en/multi-dataset/tortoise-v2")

In [ ]:
from tortoise.utils.text import split_and_recombine_text
from time import time

# Process text
text = text_to_read
if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text) #If only one piece of text (<20secs), convert it to list

In [ ]:
cd ..

In [ ]:
#Tortoise Fastest Inference from script (~2min per 20secs of audio in Colab)
#Can also be downloaded from https://huggingface.co/jbetker/tortoise-tts-v2/tree/main
import torch
import torchaudio

from TTS.tts.configs.tortoise_config import TortoiseConfig
from TTS.tts.models.tortoise import Tortoise
import IPython
import gc

config = TortoiseConfig()
model = Tortoise.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/root/.local/share/tts/tts_models--en--multi-dataset--tortoise-v2", eval=True) #Deepspeed doesn't work
model.cuda()

all_parts = []
for j, text in enumerate(texts): #Combine individuals pieces of text by processing sound one piece at a time (around 20secs each)
  # cloning a speaker
  output_dict = model.synthesize(text, config, speaker_id="voice", voice_dirs=base_path)

  #Save result
  torchaudio.save("tortoise_v2_script_"+str(j)+".wav", torch.tensor(output_dict["wav"]).squeeze(0), 24000)
  all_parts.append(torch.tensor(output_dict["wav"]).squeeze(0))
  del output_dict

  # Perform garbage collection
  gc.collect()

torchaudio.save('combined.wav', torch.cat(all_parts, dim=-1), 24000)
IPython.display.Audio('combined.wav')

Clean memory so that we can have a bit longer text to read

In [ ]:
del all_parts

In [ ]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [ ]:
import gc
# Perform garbage collection
gc.collect()

In [ ]:
from google.colab import files
files.download('/content/gdrive/MyDrive/deepfake/combined.wav')
#Individual pieces can also be found in this folder

In [ ]:
#@title Confirm that audio same length as video. If not, keep the smallest one and cut the other (or cut them both to 20secs). This is needed for wav2lip to work
#ChatGPT Prompt: Create python code that compares an audio.wav with a video.mp4 files and if the duration of one is bigger than the other,
# it cuts the largest one to be the same duration as the smallest. If any of them is bigger than 20secs then raise an error

#Needed to avoid errors with encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from moviepy.editor import VideoFileClip, AudioFileClip
import os

def compare_audio_video_duration(audio_file, video_file):
    audio = AudioFileClip(audio_file)
    video = VideoFileClip(video_file)

    audio_duration = audio.duration
    video_duration = video.duration

    # Either the video or audio should be <20 secs. If any of these is larger than that, it will be cut to the duration of the other or to 20secs.
    # Might work for up to 30secs, but not guaranteed. If only video is provided, it will keep only the first 20 secs of it.

    # if audio_duration > 20 and video_duration > 20:
    #     video = video.subclip(0, 20)
    #     video.write_videofile('input_video.mp4')
    #     audio = audio.subclip(0, 20)
    #     audio.write_audiofile('input_audio.wav')

    if audio_duration != video_duration:
        min_duration = min(audio_duration, video_duration)
        if min_duration == audio_duration:
            video = video.subclip(0, min_duration)
            video.write_videofile('input_video.mp4')
            os.rename(audio_file,'input_audio.wav')
        else:
            audio = audio.subclip(0, min_duration)
            audio.write_audiofile('input_audio.wav')
            os.rename(video_file,'input_video.mp4')

    audio.close()
    video.close()

# Example usage
compare_audio_video_duration("combined.wav", "video_full.mp4") #input_voice.mp3



---



In [ ]:
cd /content

In [ ]:
#@title <h1>Install Wav2Lip</h1>
#@markdown * Install dependencies
#@markdown * Download models
# !rm -rf /content/sample_data
# !mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O /content/Wav2Lip/checkpoints/wav2lip.pth
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
from IPython.display import clear_output
clear_output()
print("\nDone")



---



- Below implementation needs both audio and video to be of same length. Only specific extensions work (mp4 and wav)
- Target face in the input_video.mp4, must be "detectable" in ALL videoframes (So no black or blurry frames etc)
- wav2lip does not like very long and high res clips (1080p/30seconds recommended)
- 'Wav2Lip' model gives highly accurate lip-sync compared to 'Wav2Lip + GAN' but with inferior visual quality compared to the latter

Below is needed to fix an error in loading - Not added in the beginning due to conflict in dependencies

In [ ]:
!pip install librosa==0.9.1

In [ ]:
# Clear all user-defined variables except built-in ones
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

# Perform garbage collection to free up memory
import gc
gc.collect()

In [ ]:
!pip install -U numpy==2.0.0 pandas scikit-learn

In [ ]:
#@title Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/gdrive/MyDrive/deepfake/input_video.mp4' --audio '/content/gdrive/MyDrive/deepfake/input_audio.wav' --resize_factor 2

#Use --resize_factor 2 otherwise OOM error. Use resize_factor to reduce the video resolution, as there is a chance you might get better results for lower resolution videos.
# This might be related with the model which might have been trained on low resolution faces.

In [ ]:
#@title Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4') #Only after the last cell is executed this will start
files.download('/content/gdrive/MyDrive/deepfake/input_audio.wav')

In [ ]:
# #@title Delete old uploaded samples & result files, so you can start over again.
# # %rm /content/sample_data/*
# %rm /content/Wav2Lip/results/*
# from IPython.display import clear_output
# clear_output()
# print("\nDone! now press X")

# **Variations to try**


In [ ]:
# #@title Create Wav2Lip video using wav2lip.pth
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"  --resize_factor 2

In [ ]:
#@title Use more padding to include the chin region (you can manually edit pads dimensions viewing and changing the code)
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0 --resize_factor 2

In [ ]:
# #@title Play result video -  50% scaling
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML(f"""
# <video width="50%" height="50%" controls>
#       <source src="{data_url}" type="video/mp4">
# </video>""")

In [ ]:
#@title Download Result.mp4 to your computer
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4')